# Raw dataset modelling - without a lot of pre-processing

# Importing libraries and dataset

In [76]:
!pip install xgboost

  Using cached xgboost-2.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 330.3 kB/s eta 0:06:19
   ---------------------------------------- 0.0/124.9 MB 393.8 kB/s eta 0:05:18
   ---------------------------------------- 0.1/124.9 MB 819.2 kB/s eta 0:02:33
   ---------------------------------------- 0.2/124.9 MB 952.6 kB/s eta 0:02:11
   ---------------------------------------- 0.2/124.9 MB 952.6 kB/s eta 0:02:11
   ---------------------------------------- 0.3/124.9 MB 1.0 MB/s eta 0:02:05
   ---------------------------------------- 0.4/124.9 MB 1.1 MB/s eta 0:01:52
   ---------------------------------------- 0.5/124.9 MB 1.3 MB/s eta 0:01:37
   ---------------------------------------- 0.6/124.9 MB 1.3 MB/s eta 0:01:34
   ---------------------------------------- 0.6/124.9 MB 1.4 MB/s eta 0:01:27
   ---------------------------------------- 0.7/124.9 MB 1.4 MB/s eta

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packag

   ----------------------- ---------------- 74.4/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.5/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.5/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.6/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.6/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.7/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.8/124.9 MB 1.1 MB/s eta 0:00:45
   ----------------------- ---------------- 74.9/124.9 MB 1.1 MB/s eta 0:00:44
   ----------------------- ---------------- 74.9/124.9 MB 1.1 MB/s eta 0:00:44
   ------------------------ --------------- 75.0/124.9 MB 1.1 MB/s eta 0:00:44
   ------------------------ --------------- 75.1/124.9 MB 1.2 MB/s eta 0:00:44
   ------------------------ --------------- 75.2/124.9 MB 1.2 MB/s eta 0:00:44
   ------------------------ --------------- 75.3/124

In [78]:
#Importing Data and Exploratory Analysis
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from scipy.stats import chi2

#Pre-Processing and Modelling
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from statsmodels.formula.api import ols
from sklearn.ensemble import RandomForestRegressor
#import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import BayesianRidge

In [2]:
df = pd.read_csv(r'C:\Users\User\Wastewater-Treatment-Sector-Dataset\Data-Melbourne_F.csv')
df.head()

,avg_outflow,avg_inflow,total_grid,Am,BOD,COD,TN,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,year,month,day
0,2.941,2.589,175856,27.0,365.0,730.0,60.378,19.3,25.1,12.6,0.0,56,1.52,10.0,26.9,53.5,79.5,2014,1,1
1,2.936,2.961,181624,25.0,370.0,740.0,60.026,17.1,23.6,12.3,0.0,63,0.00,10.0,14.4,27.8,0.0,2014,1,2
2,2.928,3.225,202016,42.0,418.0,836.0,64.522,16.8,27.2,8.8,0.0,47,0.25,10.0,31.9,61.1,77.8,2014,1,5
3,2.928,3.354,207547,36.0,430.0,850.0,63.000,14.6,19.9,11.1,0.0,49,0.00,10.0,27.0,38.9,53.5,2014,1,6
4,2.917,3.794,202824,46.0,508.0,1016.0,65.590,13.4,19.1,8.0,0.0,65,0.00,10.0,20.6,35.2,53.5,2014,1,7


In [3]:
df = df.drop('VG', axis=1) # I'm dropping this feature because it is not described in the article 
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df.set_index('date', inplace=True)
df.drop(columns=['year', 'month', 'day'], inplace=True)

In [4]:
df.head()

,avg_outflow,avg_inflow,total_grid,Am,BOD,COD,TN,T,TM,Tm,SLP,H,PP,VV,V,VM
date,,,,,,,,,,,,,,,,
2014-01-01,2.941,2.589,175856,27.0,365.0,730.0,60.378,19.3,25.1,12.6,0.0,56,1.52,10.0,26.9,53.5
2014-01-02,2.936,2.961,181624,25.0,370.0,740.0,60.026,17.1,23.6,12.3,0.0,63,0.00,10.0,14.4,27.8
2014-01-05,2.928,3.225,202016,42.0,418.0,836.0,64.522,16.8,27.2,8.8,0.0,47,0.25,10.0,31.9,61.1
2014-01-06,2.928,3.354,207547,36.0,430.0,850.0,63.000,14.6,19.9,11.1,0.0,49,0.00,10.0,27.0,38.9
2014-01-07,2.917,3.794,202824,46.0,508.0,1016.0,65.590,13.4,19.1,8.0,0.0,65,0.00,10.0,20.6,35.2


In [5]:
fnts=12         # fontsize
fgs=(20/2, 6/2) # figure size
trs=0.6         # line transparency
plt.style.use("ggplot")

# Model Functions

In [89]:
def print_metrics(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f'{model_name} - RMSE: {rmse:.4f}, R²: {r2:.4f}')

def ols_regression(X_train, y_train, X_test):
    ols = LinearRegression()
    ols.fit(X_train, y_train)
    y_pred = ols.predict(X_test)
    return y_pred, ols

def ridge_regression(X_train, y_train, X_test, alpha):
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    return y_pred, ridge

def lasso_regression(X_train, y_train, X_test, alpha):
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    return y_pred, lasso

def pls_regression(X_train, y_train, X_test, n_components):
    pls = PLSRegression(n_components=n_components)
    pls.fit(X_train, y_train)
    y_pred = pls.predict(X_test)
    return y_pred, pls

def moving_window_ols(X, y, window_size):
    n_samples = len(y)
    preds = np.zeros(n_samples)

    for i in range(window_size, n_samples):
        X_window = X[i-window_size:i]
        y_window = y[i-window_size:i]

        ols = LinearRegression()
        ols.fit(X_window, y_window)

        preds[i] = ols.predict(X[i].reshape(1, -1))[0]

    return preds

def evaluate_moving_window(X, y, window_size):
    n_samples = len(y)
    preds = np.zeros(n_samples)
    y_true = np.zeros(n_samples)

    for i in tqdm(range(window_size, n_samples), desc=f"Iteration {window_size}"):
        X_window = X[i-window_size:i]
        y_window = y[i-window_size:i]

        # Perform OLS Regression
        model = LinearRegression()
        model.fit(X_window, y_window)

        preds[i] = model.predict(X[i].reshape(1, -1))[0]
        y_true[i] = y.iloc[i]


    # Calculate performance metrics
    rmse = np.sqrt(mean_squared_error(y_true[window_size:], preds[window_size:]))
    r2 = r2_score(y_true[window_size:], preds[window_size:])

    return rmse, r2

def moving_window_ridge(X, y, window_size, alpha):
    n_samples = len(y)
    preds = np.zeros(n_samples)

    for i in range(window_size, n_samples):
        X_window = X[i-window_size:i]
        y_window = y[i-window_size:i]

        ridge = Ridge(alpha=alpha)
        ridge.fit(X_window, y_window)

        preds[i] = ridge.predict(X[i].reshape(1, -1))[0]

    return preds

def moving_window_lasso(X, y, window_size, alpha):
    n_samples = len(y)
    preds = np.zeros(n_samples)

    for i in range(window_size, n_samples):
        X_window = X[i-window_size:i]
        y_window = y[i-window_size:i]

        lasso = Lasso(alpha=alpha)
        lasso.fit(X_window, y_window)

        preds[i] = lasso.predict(X[i].reshape(1, -1))[0]

    return preds

def fourier_series_regression(X, y, n_terms, period):
    # Flatten y if necessary
    y = y.values.flatten()
    data = pd.DataFrame(X)
    data['y'] = y

    # Creating Fourier terms (sine and cosine)
    for i in range(1, n_terms + 1):
        data[f'sin_{i}'] = np.sin(2 * np.pi * i * data.index / period)
        data[f'cos_{i}'] = np.cos(2 * np.pi * i * data.index / period)

    # Building the formula for the OLS model
    formula = 'y ~ ' + ' + '.join([f'sin_{i} + cos_{i}' for i in range(1, n_terms + 1)])

    model = ols(formula, data=data).fit()
    data['y_pred'] = model.predict(data)


    return data['y_pred']

def rf_grid_search(X_train, y_train, cv=5, scoring='r2'):
    rf = RandomForestRegressor()

    # Parameters for the grid search
    param_grid = {
        'n_estimators': [5, 10, 15, 20, 50],
        'max_depth': [1, 5, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, scoring=scoring)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_

    # Return the best estimator (fitted model) and the best parameters
    return grid_search.best_estimator_, best_params

def rolling_window_random_forest(X, y, best_params, window_size):
    n_samples = X.shape[0]

    rmse_list = []
    y_true_all = []
    y_pred_all = []

    for i in range(window_size, n_samples - window_size):
        X_train = X[i - window_size:i]
        y_train = y.iloc[i:i + window_size]  # Use iloc to access values
        X_test = X[i + window_size].reshape(1, -1)  # Single next time step as test
        y_test = y.iloc[i + window_size].reshape(1, -1)

        best_rf = RandomForestRegressor(**best_params)
        best_rf.fit(X_train, y_train)

        y_pred = best_rf.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_list.append(rmse)
        y_true_all.append(y_test[0])
        y_pred_all.append(y_pred[0])

    return y_true_all, y_pred_all, rmse_list

def knn_grid_search(X_train, y_train, cv=5, scoring='r2', verbose=1):
    # Defining the parameter grid for KNN
    param_grid = {'n_neighbors': np.arange(1, 50)}

    knn = KNeighborsRegressor()
    grid_search = GridSearchCV(knn, param_grid, cv=cv, scoring=scoring, verbose=verbose)

    grid_search.fit(X_train, y_train)
    best_knn = grid_search.best_estimator_

    return best_knn, grid_search.best_params_

def moving_window_knn(X, y, window_size, n_neighbors):
    rmse_list = []
    y_true_all = []
    y_pred_all = []

    for i in range(len(X) - window_size):
        X_train = X[i:i + window_size]
        y_train = y.iloc[i:i + window_size]  # Use iloc to access values
        X_test = X[i + window_size].reshape(1, -1)  # Single next time step as test
        y_test = y.iloc[i + window_size].reshape(1, -1)

        knn = KNeighborsRegressor(n_neighbors=n_neighbors)
        knn.fit(X_train, y_train)

        y_pred = knn.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_list.append(rmse)
        y_true_all.append(y_test[0])
        y_pred_all.append(y_pred[0])

    return y_true_all, y_pred_all, rmse_list

def huber_regression(X_train, y_train):

    huber = HuberRegressor()

    huber.fit(X_train, y_train)
    preds = huber.predict(X_test)

    return preds, huber

def moving_window_huber(X, y, window_size):
    n_samples = X.shape[0]
    rmse_list = []
    y_true_all = []
    y_pred_all = []

    for i in range(window_size, n_samples):  # Start at window_size to avoid empty training data
        X_train = X[i - window_size:i]  # Training data
        y_train = y.iloc[i - window_size:i]  # Training labels
        X_test = X[i].reshape(1, -1)  # Current time step for testing
        y_test = y.iloc[i].reshape(1, -1)  # Current label

        huber = HuberRegressor()
        huber.fit(X_train, y_train)

        y_pred = huber.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_list.append(rmse)
        y_true_all.append(y_test[0])
        y_pred_all.append(y_pred[0])

    return y_true_all, y_pred_all, rmse_list

def bayesian_regression(X_train, y_train):

    bayes_ridge = BayesianRidge()

    bayes_ridge.fit(X_train, y_train)
    preds = bayes_ridge.predict(X_test)

    return preds, bayes_ridge

def moving_window_bayesian(X, y, window_size):
    n_samples = X.shape[0]  # Number of samples
    rmse_list = []
    y_true_all = []
    y_pred_all = []

    for i in range(window_size, n_samples):
        X_train = X[i - window_size:i]
        y_train = y.iloc[i - window_size:i]  # Training labels
        X_test = X[i].reshape(1, -1)  # Current time step for testing
        y_test = y.iloc[i].reshape(1, -1)  # Current label

        bayes_ridge = BayesianRidge()
        bayes_ridge.fit(X_train, y_train)

        y_pred = bayes_ridge.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        rmse_list.append(rmse)
        y_true_all.append(y_test[0])
        y_pred_all.append(y_pred[0])

    return y_true_all, y_pred_all, rmse_list

# Creating the dataframe used in the research

In [7]:
df_prep = df.dropna()

In [8]:
df_prep.index.duplicated().sum()

0

Predictor and output datasets

In [9]:
df_prep = df_prep.drop("total_grid", axis=1)  # Input features (excluding energy consumption)
y = df["total_grid"]  # Target variable (energy consumption)

# Pre-processing (very simple)

In [10]:
scaler = MinMaxScaler()
df_prep_scaled = scaler.fit_transform(df_prep)

# Dividing into train and test datasets

In [11]:
df_Y = y.copy()
X_train, X_test, y_train, y_test = train_test_split(df_prep, df_Y, test_size=0.2, shuffle=False)

# Modelling

## OLS

In [12]:
y_pred_ols, ols_model = ols_regression(X_train, y_train, X_test)
print_metrics(y_test, y_pred_ols, "OLS Regression")

OLS Regression - RMSE: 41486.0951, R²: -0.1616


## Moving Window OLS

In [14]:
window_sizes = range(1, 30)
results = []

for size in window_sizes:
    rmse, r2 = evaluate_moving_window(df_prep_scaled, df_Y, size)
    results.append((size, rmse, r2))

best_window = min(results, key=lambda x: x[1])
print(f'\n Best Window Size: {best_window[0]}, RMSE: {best_window[1]:.4f}, R²: {best_window[2]:.4f}')

Iteration 29: 100%|██████████| 1353/1353 [00:01<00:00, 965.41it/s]


 Best Window Size: 1, RMSE: 40240.4610, R²: 0.1845


In [15]:
y_pred_moving_window_ols = moving_window_ols(df_prep_scaled, df_Y, best_window[0])
print_metrics(df_Y[best_window[0]:], y_pred_moving_window_ols[best_window[0]:], "Moving Window OLS Regression")

Moving Window OLS Regression - RMSE: 40240.4610, R²: 0.1845


## Ridge 

In [39]:
alpha_values = np.linspace(1000, 2000, 100)

ridge = Ridge()
ridge_params = {'alpha': alpha_values}
ridge_grid_search = GridSearchCV(ridge, ridge_params, scoring='r2', cv=5)
ridge_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': array([1000.        , 1010.1010101 , 1020.2020202 , 1030.3030303 ,
       1040.4040404 , 1050.50505051, 1060.60606061, 1070.70707071,
       1080.80808081, 1090.90909091, 1101.01010101, 1111.11111111,
       1121.21212121, 1131.31313131, 1141.41414141, 1151.51515152,
       1161.61616162, 1171.71717172, 1181.81818182, 1191.91919192,
       1202.02020202, 1212....
       1727.27272727, 1737.37373737, 1747.47474747, 1757.57575758,
       1767.67676768, 1777.77777778, 1787.87878788, 1797.97979798,
       1808.08080808, 1818.18181818, 1828.28282828, 1838.38383838,
       1848.48484848, 1858.58585859, 1868.68686869, 1878.78787879,
       1888.88888889, 1898.98989899, 1909.09090909, 1919.19191919,
       1929.29292929, 1939.39393939, 1949.49494949, 1959.5959596 ,
       1969.6969697 , 1979.7979798 , 1989.8989899 , 2000.        ])},
             scoring='r2')

In [40]:
best_ridge_alpha = ridge_grid_search.best_params_['alpha']
ridge_best_model = ridge_grid_search.best_estimator_
y_pred_ridge_best = ridge_best_model.predict(X_test)
print(f'Best Ridge Alpha: {best_ridge_alpha}')
print_metrics(y_test, y_pred_ridge_best, "Best Ridge Regression")

Best Ridge Alpha: 1000.0
Best Ridge Regression - RMSE: 40493.6181, R²: -0.1067


In [41]:
y_pred_ridge, ridge_model = ridge_regression(X_train, y_train, X_test, best_ridge_alpha)
print_metrics(y_test, y_pred_ridge, "Ridge Regression")

Ridge Regression - RMSE: 40493.6181, R²: -0.1067


## Moving Window Ridge

In [48]:
y_pred_ridge = moving_window_ridge(df_prep_scaled, df_Y, window_size=7, alpha=best_ridge_alpha)
print_metrics(df_Y[7:], y_pred_ridge[7:], "Ridge Regression with Moving OLS")

Ridge Regression with Moving OLS - RMSE: 37103.8785, R²: 0.3027


## Lasso

In [53]:
alpha_values = np.linspace(1000,2000, 50)
lasso = Lasso(max_iter=10000)
lasso_params = {'alpha': alpha_values}
lasso_grid_search = GridSearchCV(lasso, lasso_params, scoring='r2', cv=5)
lasso_grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(max_iter=10000),
             param_grid={'alpha': array([1000.        , 1020.40816327, 1040.81632653, 1061.2244898 ,
       1081.63265306, 1102.04081633, 1122.44897959, 1142.85714286,
       1163.26530612, 1183.67346939, 1204.08163265, 1224.48979592,
       1244.89795918, 1265.30612245, 1285.71428571, 1306.12244898,
       1326.53061224, 1346.93877551, 1367.34693878, 1387.75510204,
       14...
       1489.79591837, 1510.20408163, 1530.6122449 , 1551.02040816,
       1571.42857143, 1591.83673469, 1612.24489796, 1632.65306122,
       1653.06122449, 1673.46938776, 1693.87755102, 1714.28571429,
       1734.69387755, 1755.10204082, 1775.51020408, 1795.91836735,
       1816.32653061, 1836.73469388, 1857.14285714, 1877.55102041,
       1897.95918367, 1918.36734694, 1938.7755102 , 1959.18367347,
       1979.59183673, 2000.        ])},
             scoring='r2')

In [54]:
best_lasso_alpha = lasso_grid_search.best_params_['alpha']
lasso_best_model = lasso_grid_search.best_estimator_
y_pred_lasso_best = lasso_best_model.predict(X_test)
print(f'Best Lasso Alpha: {best_lasso_alpha}')
print_metrics(y_test, y_pred_lasso_best, "Best Lasso Regression")

Best Lasso Alpha: 1000.0
Best Lasso Regression - RMSE: 41090.8465, R²: -0.1396


In [55]:
y_pred_lasso, lasso_model = lasso_regression(X_train, y_train, X_test, best_lasso_alpha)
print_metrics(y_test, y_pred_lasso, "Lasso Regression")

Lasso Regression - RMSE: 41090.8465, R²: -0.1396


## Moving Window Lasso

In [59]:
y_pred_lasso = moving_window_lasso(df_prep_scaled, df_Y, window_size=30, alpha=best_lasso_alpha)
print_metrics(df_Y[30:], y_pred_lasso[30:], "Lasso Regression with Moving OLS")

Lasso Regression with Moving OLS - RMSE: 39066.8904, R²: 0.2014


## PLS 

In [64]:
y_pred_pls, pls_model = pls_regression(X_train, y_train, X_test, 5)
print_metrics(y_test, y_pred_pls, "PLS Regression")

PLS Regression - RMSE: 41523.4826, R²: -0.1637


## Adaboost

In [68]:
# Initialize the base model for AdaBoost
base_model = DecisionTreeRegressor(max_depth=5)

# Initialize AdaBoost with the base model
adaboost = AdaBoostRegressor(base_model)
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.01, 0.1]
}

In [69]:
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, scoring='r2')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score (R²): {best_score}")


Best Parameters: {'learning_rate': 0.001, 'n_estimators': 100}
Best Score (R²): -0.10266341100158735


In [70]:
best_model = grid_search.best_estimator_
y_pred_ada = best_model.predict(X_test)
print_metrics(y_test, y_pred_ada, "AdaBoost")

AdaBoost - RMSE: 38783.0165, R²: -0.0151


## XGBoost

Perform later

In [79]:
# xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# param_grid = {
#    'max_depth': [3, 6, 9],
#    'learning_rate': [0.01, 0.1, 0.2],
#    'n_estimators': [50, 100, 150, 200],
#    'subsample': [0.8, 0.9, 1.0]
#}

## Random Forest

In [80]:
best_model, best_params = rf_grid_search(X_train, y_train)
print("Best parameters:", best_params)

Best parameters: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 20}


In [81]:
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, min_samples_leaf=4, min_samples_split=10,
                      n_estimators=20)

In [82]:
y_pred_rf = best_rf.predict(X_test)
print_metrics(y_test, y_pred_rf, "Random Forest")

Random Forest - RMSE: 39244.2778, R²: -0.0394


## Moving Window Random Forest

In [92]:
y_true, y_pred_rf, rmse_list = rolling_window_random_forest(df_prep_scaled, df_Y, best_params, 7)
average_rmse = np.mean(rmse_list)
r2 = r2_score(y_true, y_pred_rf)
print(f'Average RMSE with moving window: {average_rmse}')
print(f'R^2 with moving window: {r2}')

Average RMSE with moving window: 28850.34955096074
R^2 with moving window: 0.2816476494927196


## KNN

In [90]:
best_knn, best_params_knn = knn_grid_search(X_train, y_train)
print("Best parameters:", best_params_knn)

y_pred_knn = best_knn.predict(X_test)
print_metrics(y_test, y_pred_knn, "KNN")

Fitting 5 folds for each of 49 candidates, totalling 245 fits
Best parameters: {'n_neighbors': 49}
KNN - RMSE: 38611.6102, R²: -0.0062


## Moving Window KNN

In [98]:
y_true, y_pred_knn, rmse_list = moving_window_knn(df_prep_scaled, df_Y, 30, 29)
average_rmse = np.mean(rmse_list)
r2 = r2_score(y_true, y_pred_knn)
print(f'Average RMSE with moving window: {average_rmse}')
print(f'R^2 with moving window: {r2}')

Average RMSE with moving window: 30944.904585798817
R^2 with moving window: 0.18462666758083812


## Huber Regression

In [99]:
y_pred_huber, model = huber_regression(X_train, y_train)
print_metrics(y_test, y_pred_huber, "Huber Regression")

Huber Regression - RMSE: 44162.1314, R²: -0.3163


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Moving Window Huber Regression

In [102]:
y_true, y_pred_huber, rmse_list = moving_window_huber(df_prep_scaled, df_Y, 7)
average_rmse = np.mean(rmse_list)
r2 = r2_score(y_true, y_pred_huber)
print(f'Average RMSE with moving window: {average_rmse}')
print(f'R^2 with moving window: {r2}')

Average RMSE with moving window: 28595.217020297863
R^2 with moving window: 0.2923521962003963


## Bayesian Regression

In [104]:
y_pred_bayes, model_bayes = bayesian_regression(X_train, y_train)
print_metrics(y_test, y_pred_bayes, "Bayesian Regression")

Bayesian Regression - RMSE: 40547.8431, R²: -0.1096


## Moving Window Bayesian Regression

In [107]:
y_true, y_pred_bayes, rmse_list = moving_window_bayesian(df_prep_scaled, df_Y, 7)
average_rmse = np.mean(rmse_list)
r2 = r2_score(y_true, y_pred_bayes)
print(f'Average RMSE with moving window: {average_rmse}')
print(f'R^2 with moving window: {r2}')

Average RMSE with moving window: 28582.948363119896
R^2 with moving window: 0.3027043761835335
